# Introduccion

- Trabajas para la compañía minera OilyGiant. Tu tarea es encontrar el mejor lugar para un nuevo pozo. Pasos para elegir la ubicación:
1. Recolecta los parámetros del pozo de petróleo en la región seleccionada: calidad del petróleo y volumen de reservas;
2. Construye un modelo para predecir el volumen de reservas en los nuevos pozos;
3. Selecciona los pozos de petróleo con los valores estimados más altos;
4. Elige la región con el mayor beneficio total para los pozos de petróleo seleccionados.
- Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

# Contenido
* [1. Introducción]
* [2. Contenidos]
* [3. Inicialización]
    * [3.1 Cargar las librerias]
    * [3.2 Cargar los datos]
* [4. Generacion de modelos]
    * [4.1 Segmentacion de datos]
    * [4.2 Entrenamiento de modelos]
    * [4.3 Guardar las predicciones]
    * [4.4 Mostrar RECM y Volumen promedio]
    * [4.5 Analisis del mejor modelo]
* [5. Calculo de ganancias]

# Inicializacion

## Cargar las librerias

In [ ]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, r2_score, roc_auc_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

## Cargar los datos

In [ ]:
region1_data = pd.read_csv('/datasets/geo_data_0.csv')
region2_data = pd.read_csv('/datasets/geo_data_1.csv')
region3_data = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
display(region1_data.shape)
display(region2_data.shape)
display(region3_data.shape)

(100000, 5)

(100000, 5)

(100000, 5)

In [ ]:
display(region1_data.info())
display(region2_data.info())
display(region3_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [ ]:
# Muestra de la region 1
region1_data.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
# Muestra de la region 2
region2_data.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [ ]:
# Muestra de la region 3
region3_data.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


## Correccion de datos

In [ ]:
region1_data = region1_data.drop(['id'], axis=1)
region2_data = region2_data.drop(['id'], axis=1)
region3_data = region3_data.drop(['id'], axis=1)

# Generacion de modelos

## Segmentacion de datos

In [ ]:
# Segmentamos los dataframes en conjunto de entrenamiento y de validacion.
region1_train, region1_val, region1_target_train, region1_target_val = train_test_split(region1_data.drop('product', axis=1),
                                                          region1_data['product'],
                                                          test_size=0.25, random_state=12345)

region2_train, region2_val, region2_target_train, region2_target_val = train_test_split(region2_data.drop('product', axis=1),
                                                          region2_data['product'],
                                                          test_size=0.25, random_state=12345)

region3_train, region3_val, region3_target_train, region3_target_val = train_test_split(region3_data.drop('product', axis=1),
                                                          region3_data['product'],
                                                          test_size=0.25, random_state=12345)

## Entrenamiento de modelos

In [ ]:
# Se entrena un modelo de regresion lineal para las tres regiones
model1 = LinearRegression()
model1.fit(region1_train, region1_target_train)
region1_predictions = model1.predict(region1_val)

model2 = LinearRegression()
model2.fit(region2_train, region2_target_train)
region2_predictions = model2.predict(region2_val)

model3 = LinearRegression()
model3.fit(region3_train, region3_target_train)
region3_predictions = model3.predict(region3_val)

## Guardar las predicciones

In [ ]:
# Se crean 3 dataframes con las predicciones de las tres regiones con los modelos entrenados anteriormente
region1_predictions_df = pd.DataFrame({'predictions': region1_predictions, 'target': region1_target_val})
region2_predictions_df = pd.DataFrame({'predictions': region2_predictions, 'target': region2_target_val})
region3_predictions_df = pd.DataFrame({'predictions': region3_predictions, 'target': region3_target_val})

In [ ]:
# Se separan los dataframes con 500 muestras aleatorias
region1_predictions_500 = region1_predictions_df.sample(n=500, random_state=12345)
region2_predictions_500 = region2_predictions_df.sample(n=500, random_state=12345)
region3_predictions_500 = region3_predictions_df.sample(n=500, random_state=12345)

In [ ]:
# Se separan los dataframes con los 200 numeros mas grandes
top_wells_region1 = region1_predictions_500.nlargest(200, "predictions")
top_wells_region2 = region2_predictions_500.nlargest(200, "predictions")
top_wells_region3 = region3_predictions_500.nlargest(200, "predictions")

## Mostrar RECM y Volumen promedio

In [ ]:
# Volumen promedio y RECM del modelo para la región 1:
mean_volume_region1 = region1_predictions_df['predictions'].mean()
rmse_region1 = mean_squared_error(region1_predictions_500['target'], region1_predictions_500['predictions'], squared=False)
print("Volumen promedio de las reservas previstas: {:.2f}".format(region1_predictions_500['predictions'].mean()))
print(f'RECM del modelo: {rmse_region1:.2f}')

Volumen promedio de las reservas previstas: 92.93
RECM del modelo: 37.58


In [ ]:
# Volumen promedio y RECM del modelo para la región 2:
mean_volume_region2 = region2_predictions_df['predictions'].mean()
rmse_region2 = mean_squared_error(region2_predictions_500['target'], region2_predictions_500['predictions'], squared=False)
print("Volumen promedio de las reservas previstas: {:.2f}".format(region2_predictions_500['predictions'].mean()))
print(f'RECM del modelo: {rmse_region2:.2f}')

Volumen promedio de las reservas previstas: 71.84
RECM del modelo: 0.90


In [ ]:
# Volumen promedio y RECM del modelo para la región 3:
mean_volume_region3 = region3_predictions_df['predictions'].mean()
rmse_region3 = mean_squared_error(region3_predictions_500['target'], region3_predictions_500['predictions'], squared=False)
print("Volumen promedio de las reservas previstas: {:.2f}".format(region3_predictions_500['predictions'].mean()))
print(f'RECM del modelo: {rmse_region3:.2f}')

Volumen promedio de las reservas previstas: 94.73
RECM del modelo: 40.61


## Analisis del mejor modelo

- Como podemos observar la mejor region es la region 3 debido a que su volumen promedio de las reservas es el mas alto con un valor de 94.73, aunque no por mucho ya que la region 1 cuenta con un valor de 92.93. Esta decision se toma basada en maximizar la cantidad total de petroleo que se extraera por ende el mejor pozo debera ser en la region 3.

# Calculo de ganancias

In [ ]:
# Se almacenan todos los valores clave para los cálculos en variables separadas.
budget = 100000000
price_per_barrel = 4.5
max_risk = 0.025
n_wells = 200
revenue = 4500
cost_per_well = budget / n_wells
revenue_per_well1 = price_per_barrel * (mean_volume_region1 / 1000)
revenue_per_well2 = price_per_barrel * (mean_volume_region2 / 1000)
revenue_per_well3 = price_per_barrel * (mean_volume_region3 / 1000)

In [ ]:
# Volumen minimo de pozos
min_volume = cost_per_well / revenue
print(f'El minimo de pozos por region para no obtener perdidas es de: {min_volume:.2f} miles de barriles')

El minimo de pozos por region para no obtener perdidas es de: 111.11 miles de barriles


In [ ]:
# Se pone un condicional para cada region para determinar si tiene suficientes reservas para desarrollar un pozo sin perdidas
if mean_volume_region1 >= min_volume:
    print("La región 1 tiene suficientes reservas para desarrollar un nuevo pozo sin pérdidas.")
else:
    print("La región 1 no tiene suficientes reservas para desarrollar un nuevo pozo sin pérdidas.")

if mean_volume_region2 >= min_volume:
    print("La región 2 tiene suficientes reservas para desarrollar un nuevo pozo sin pérdidas.")
else:
    print("La región 2 no tiene suficientes reservas para desarrollar un nuevo pozo sin pérdidas.")

if mean_volume_region3 >= min_volume:
    print("La región 3 tiene suficientes reservas para desarrollar un nuevo pozo sin pérdidas.")
else:
    print("La región 3 no tiene suficientes reservas para desarrollar un nuevo pozo sin pérdidas.")

La región 1 no tiene suficientes reservas para desarrollar un nuevo pozo sin pérdidas.
La región 2 no tiene suficientes reservas para desarrollar un nuevo pozo sin pérdidas.
La región 3 no tiene suficientes reservas para desarrollar un nuevo pozo sin pérdidas.


# Funcion para calcular la ganancia

In [ ]:
# Se define una funcion para calcular el beneficio de cada region.
def calculate_profit1(total_wells, cost_per_well, revenue_per_well, predicted_volume):
    total_revenue = revenue_per_well * predicted_volume.sum() * 1000
    total_cost = cost_per_well * total_wells
    profit =  total_revenue - total_cost
    return profit

In [ ]:
profit_region1 = calculate_profit1(top_wells_region1.shape[0], cost_per_well, revenue_per_well1, top_wells_region1["predictions"])
profit_region2 = calculate_profit1(top_wells_region2.shape[0], cost_per_well, revenue_per_well2, top_wells_region2["predictions"])
profit_region3 = calculate_profit1(top_wells_region3.shape[0], cost_per_well, revenue_per_well3, top_wells_region3["predictions"])

In [ ]:
# Beneficio para la region 1
print(f'El beneficio para la region 1 es de: {profit_region1:.2f} miles de dolares')

El beneficio para la region 1 es de: -90272829.56 miles de dolares


In [ ]:
# Beneficio para la region 2
print(f'El beneficio para la region 2 es de: {profit_region2:.2f} miles de dolares')

El beneficio para la region 2 es de: -92591599.28 miles de dolares


In [ ]:
# Beneficio para la region 3
print(f'El beneficio para la region 3 es de: {profit_region3:.2f} miles de dolares')

El beneficio para la region 3 es de: -90180424.62 miles de dolares


- Como podemos observar el beneficio mas alto es el de la region 2 seguido por el de la region 1. Entonces si tuviera que escoger una region basandome exclusivamente en el beneficio total escogeria la region 2.

# Bootstrap

In [ ]:
# Se define la funcion para realizar bootstrap a 1000 muestras
def bootstrap_profit(region_predictions_df, revenue_per_well, cost_per_well, n_wells, n_simulations=1000, alpha=0.05):
    profits = []
    for i in range(n_simulations):

        # es importante no colocar semilla para que en cada iteración aparezca una muestra distinta

        top_500 = region_predictions_df.sample(n=500, replace=True)

        # tomamos los 200 mejores en predicción

        top_200_pred = top_500.nlargest(200, "predictions")

        predicted_volume = top_200_pred['predictions']
        total_revenue = 4.5 * predicted_volume.sum() * 1000
        total_cost = budget
        profit = total_revenue - total_cost

        profits.append(profit)
         # Calculate mean and confidence interval
        mean_profit = np.mean(profits)
        lower_ci = np.percentile(profits, (alpha/2)*100)
        upper_ci = np.percentile(profits, (1-alpha/2)*100)

        # Calculate loss probability
        loss_probability = sum([1 for profit in profits if profit < 0]) / len(profits)
        loss_percentage = loss_probability * 100

    return mean_profit, lower_ci, upper_ci, loss_percentage

In [ ]:
# Se define la funcion para realizar bootstrap a 1000 muestras
def bootstrap_profit(region_predictions_df, n_simulations=1000, alpha=0.05):
    profits = []
    for i in range(n_simulations):

        # es importante no colocar semilla para que en cada iteración aparezca una muestra distinta

        top_500 = region_predictions_df.sample(n=500, replace=True)

        # tomamos los 200 mejores en predicción

        top_200_pred = top_500.nlargest(200, "predictions")

        predicted_volume = top_200_pred['target']
        total_revenue = 4.5 * predicted_volume.sum() * 1000
        total_cost = budget
        profit = total_revenue - total_cost

        profits.append(profit)
         # Calculate mean and confidence interval
        mean_profit = np.mean(profits)
        lower_ci = np.percentile(profits, (alpha/2)*100)
        upper_ci = np.percentile(profits, (1-alpha/2)*100)

        # Calculate loss probability
        loss_probability = sum([1 for profit in profits if profit < 0]) / len(profits)
        loss_percentage = loss_probability * 100

    return mean_profit, lower_ci, upper_ci, loss_percentage

In [ ]:
# Region 1
bootstrap_profit(region1_predictions_df)

(3969756.4334402536, -1190995.9659775821, 9142530.09961154, 6.6000000000000005)

In [ ]:
# Region 2
bootstrap_profit(region2_predictions_df)

(4561332.429502339, 702245.1674567886, 8451193.680997044, 1.0999999999999999)

In [ ]:
# Region 3
bootstrap_profit(region3_predictions_df)

(3923835.0497809616, -1278518.6999147288, 9055778.57150573, 8.3)

# Conclusion

- Finalmente con la tecnica boostrapping podemos ver que el beneficio promedio mas alto es para la region 2 y con nuestros analisis anteriores la region 3 fue la mejor con el volumen promedio esperado. Entonces si tuviera que escoger una region para la creacion de pozos seria la region 2 debido a que tras realizar bootstrapping nos da que el beneficio promedio mas alto es para la region 2.